In [2]:
import random

import cv2
import numpy as np

In [3]:
img = cv2.imread("./images/chair.jpg")

In [4]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [5]:
_ = cv2.imwrite("./images/gray.jpg", gray)

In [6]:
def sp_noise(image, prob):
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]

    return output

In [7]:
noise_gray_img = sp_noise(gray, 0.05)

In [8]:
_ = cv2.imwrite("./images/noise_gray_img.jpg", noise_gray_img)

In [9]:
denoised_image = cv2.fastNlMeansDenoising(noise_gray_img, None, 70, 5, 20)

In [10]:
_ = cv2.imwrite("./images/denoised_image.jpg", denoised_image)

In [11]:
ret, thresh_img = cv2.threshold(denoised_image, 200, 255, cv2.THRESH_BINARY)

In [12]:
contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [13]:
img_contours = np.zeros(img.shape)

In [14]:
_ = cv2.drawContours(img_contours, contours, -1, (0,255,0), 3)

In [15]:
_ = cv2.imwrite("./images/contoured_image.jpg", img_contours)

## Detection

In [22]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]

COLORS = np.random.uniform(0, 255, size = (len(CLASSES), 3))

In [17]:
net = cv2.dnn.readNetFromCaffe("./KPI_Data_Science/MobileNetSSD_deploy.prototxt.txt", "./KPI_Data_Science/MobileNetSSD_deploy.caffemodel")

In [18]:
blob = cv2.dnn.blobFromImage(cv2.resize(img_contours, (300,300)), 0.007843, (300,300), 127.5)

In [19]:
net.setInput(blob)
detections = net.forward()

In [25]:
h, w = img_contours.shape[:2]

confidence = detections[0, 0, 0, 2]

if confidence > 0.2:
    idx = int(detections[0,0, 0, 1])
    box = detections[0, 0, 0, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")

    label = "{}: {:.2f}%".format(CLASSES[idx], confidence*100)
    print("[INFO] {}".format(label))
    cv2.rectangle(img_contours, (startX, startY), (endX, endY), COLORS[idx], 2)

    y = startY-15 if startY-15 > 15 else startY+15
    cv2.putText(img_contours, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

cv2.imwrite("./images/detected_image.jpg", img_contours)

[INFO] chair: 100.00%


True